In [1]:
import numpy as np

# Einstein Summation

This notebook is used to help me get a feel of Einstein summation notation. As an example, I am trying to construct the categorical cross-entropy (negative log-likelihood multi-class) loss gradient with respect to the inputs to softmax for a mini-batch.

## Symbolically

Before diving into the Einstein summation notation, we need to know what we are trying to construct (and where it comes from). We are trying to compute

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} $$

Using chain rule, we want to break up the computation into two parts: the gradient of the negative log-likelihood multiclass loss with respect to its inputs and the gradient of the softmax function with respect to its inputs. That is,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} =\frac{\partial A^L}{\partial Z^L}\frac{\partial \mathrm{NLLM}}{\partial A^L} $$

### Softmax Function

First, we consider the softmax function. The softmax function is defined for a vector $Z^L$ with dimensions $n^L \times b$ (the output of a neural network with $n^L$ neurons in the final layer $L$ for a batch of size $b$) as the following:

$$ \mathrm{softmax}\left(Z^L\right) = \begin{pmatrix}
    \exp{z_{11}^L}/\sum_{m=1}^{n^L}\exp{z_{m1}^L} & \cdots & \exp{z_{1b}^L}/\sum_{m=1}^{n^L}\exp{z_{mb}^L} \\
    \vdots & \ddots & \vdots \\
    \exp{z_{n^L1}^L}/\sum_{m=1}^{n^L}\exp{z_{m1}^L} & \cdots & \exp{z_{n^Lb}^L}/\sum_{m=1}^{n^L}\exp{z_{mb}^L}
\end{pmatrix} $$

For simplicity, let $\mathrm{softmax}\left(Z^L\right) = A^L$. Thus, elementwise, this is the same as

$$ a_{ij}^L = \frac{\exp{z_{ij}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}} $$

### Softmax Gradient

Now, we want to construct the gradient of this function with respect to the input matrix. This is a matrix by matrix derivative. We should expect a $4$-dimensional tensor.

#### Single Element Derivative

Let's consider the derivative of a single element of softmax $a_{ij}^L$ with respect to a single input $z_{k\ell}^L$. That is, we are looking for $\partial a_{ij}^L / \partial z_{k\ell}^L$. Writing this out in terms of the softmax:

$$ \frac{\partial}{\partial z_{k\ell}}\left(\frac{\exp{z_{ij}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}}\right) $$

Using the quotient rule,

$$ \frac{\partial a_{ij}^L}{\partial z_{k\ell}^L} = \frac{\frac{\partial}{\partial z_{k\ell}^L}\left(\exp{z_{ij}^L}\right) \cdot \left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) - \left(\exp{z_{ij}^L}\right) \cdot \frac{\partial}{\partial z_{k\ell}^L}\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right)}{\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right)^2} $$

First, let's focus on the first derivative in the numerator,

$$ \frac{\partial}{\partial z_{k\ell}^L}\left(\exp{z_{ij}^L}\right) $$

This will always be zero, unless $i = k$ and $j = \ell$. Then, the derivative will be $\exp{z_{k\ell}^L}$. We will represent this using an indicator variable,

$$ \frac{\partial}{\partial z_{k\ell}^L}\left(\exp{z_{ij}^L}\right) = \exp{z_{k\ell}^L} \cdot \mathbb{1}_{i=k, j=\ell}$$

Next, let's consider the last derivative in the numerator,

$$ \frac{\partial}{\partial z_{k\ell}^L} \left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) $$

Remember, $k \in \left\{1, \ldots, n^L\right\}$ and this is also the range of $m$. Therefore, no matter what $k$ is, as long as $j = \ell$, the derivative will be $\exp{z_{k\ell}^L}$. Otherwise, the derivative will be zero. We will represent this using an indicator variable,

$$ \frac{\partial}{\partial z_{k\ell}^L} \left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) = \exp{z_{k\ell}^L} \cdot \mathbb{1}_{j=\ell} $$

Putting this all together,

$$ \frac{\partial a_{ij}^L}{\partial z_{k\ell}^L} = \frac{\left(\exp{z_{k\ell}^L} \cdot \mathbb{1}_{i=k, j=\ell}\right) \cdot \left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) - \left(\exp{z_{ij}^L}\right) \cdot \left(\exp{z_{k\ell}^L} \cdot \mathbb{1}_{j=\ell}\right)}{\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right)^2} $$

With some simplification,

\begin{align*}
    \frac{\partial a_{ij}^L}{\partial z_{k\ell}^L} &= \exp{z_{k\ell}^L} \cdot \left(\frac{\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) \cdot \mathbb{1}_{i=k} - \exp{z_{ij}^L}}{\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right)^2}\right) \cdot \mathbb{1}_{j=\ell} \\
    &= \frac{\exp{z_{k\ell}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}} \cdot \left(\frac{\left(\sum_{m=1}^{n^L} \exp{z_{mj}^L}\right) \cdot \mathbb{1}_{i=k} - \exp{z_{ij}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}}\right) \cdot \mathbb{1}_{j=\ell} \\
    &= \frac{\exp{z_{k\ell}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}} \cdot \left(\mathbb{1}_{i=k} - \frac{\exp{z_{ij}^L}}{\sum_{m=1}^{n^L} \exp{z_{mj}^L}}\right) \cdot \mathbb{1}_{j=\ell} \\
    &= a_{k\ell}^L \cdot \left(\mathbb{1}_{i=k} - a_{ij}^L\right) \cdot \mathbb{1}_{j=\ell}
\end{align*}

If you are more familiar with piecewise notation, this is equivalent to,

$$ \frac{\partial a_{ij}^L}{\partial z_{k\ell}^L} = \begin{cases}
    a_{ij}^L \cdot \left(1 - a_{ij}^L\right) & \mathrm{if}\ i = k\ \mathrm{and}\ j = \ell \\
    - a_{kj}^L \cdot a_{ij}^L & \mathrm{if}\ i \neq k\ \mathrm{and}\ j = \ell \\
    0 & \mathrm{if}\ i \neq k\ \mathrm{and}\ j \neq \ell \\
\end{cases} $$

#### Complete Gradient

Using the results from the previous section, we can easily construct a $4$-dimensional tensor of all the derivatives. However, it is important to note that the gradients are all zero when $j \neq \ell$. Intuitively, this is stating that the results from one batch are completely unrelated to another batch (which should be clear by design). Therefore, we will simply drop this component of our tensor and we are left with a $3$-dimensional tensor (which is easier to visualize and define). 

Using the definition from the previous section, we can simplify it a little,

$$ \frac{\partial a_{ij}^L}{\partial z_{kj}^L} = \begin{cases}
    a_{ij}^L \cdot \left(1 - a_{ij}^L\right) & \mathrm{if}\ i = k \\
    - a_{kj}^L \cdot a_{ij}^L & \mathrm{if}\ i \neq k
\end{cases} $$

Let's consider what this matrix looks like for a single batch. That is, assume that $j=1$ and the $b=1$. We can simplify our result even further by dropping the $j$ index altogether,

$$ \frac{\partial a_{i}^L}{\partial z_{k}^L} = \begin{cases}
    a_{i}^L \cdot \left(1 - a_{i}^L\right) & \mathrm{if}\ i = k \\
    - a_{k}^L \cdot a_{i}^L & \mathrm{if}\ i \neq k
\end{cases} $$

Now we can construct this matrix (remember, we are assuming $j=1$ and $b=1$),

$$ \frac{\partial A^L}{\partial Z^L} = \begin{pmatrix}
    a_1^L \left(1 - a_1^L\right) & -a_2^L a_1^L & \cdots & -a_{n^L}^L a_1^L \\
    -a_1^L a_2^L & a_2^L \left(1 - a_2^L\right) & \cdots & -a_{n^L}^L a_2^L \\
    \vdots & \vdots & \ddots & \vdots \\
    -a_1^L a_{n^L}^L & -a_2^L a_{n^L}^L & \cdots & a_{n^L}^L \left(1 - a_{n^L}^L\right)
\end{pmatrix} $$

If our assumptions do not hold (which is definitely true for batch updates), this matrix simply duplicates into the third dimension for every $j \in \left\{1, \ldots, b\right\}$. That is,

$$ \left(\frac{\partial A^L}{\partial Z^L}\right)_{::j} = \begin{pmatrix}
    a_{1j}^L \left(1 - a_{1j}^L\right) & -a_{2j}^L a_{1j}^L & \cdots & -a_{n^Lj}^L a_{1j}^L \\
    -a_{1j}^L a_{2j}^L & a_{2j}^L \left(1 - a_{2j}^L\right) & \cdots & -a_{n^Lj}^L a_{2j}^L \\
    \vdots & \vdots & \ddots & \vdots \\
    -a_{1j}^L a_{n^Lj}^L & -a_{2j}^L a_{n^Lj}^L & \cdots & a_{n^Lj}^L \left(1 - a_{n^Lj}^L\right)
\end{pmatrix} $$

(Where the subscript $::j$ for the gradient represents taking the $j$ matrix from the third dimension.)

### Negative Log-Likelihood Multiclass Function

Now we can consider the categorical cross-entropy function. The categorical cross-entropy function is defined for a prediction vector $A^L$ and an expected vector $Y$ both with dimensions $n^L \times b$ (the output of a neural network with $n^L$ neurons in the final layer $L$ for a batch of size $b$) as the following:

$$ \mathrm{NLLM}(A^L, Y) = - \sum_{i=1}^{n^L}\sum_{j=1}^{b} y_{ij} \cdot \log a_{ij}^L $$

### Negative Log-Likelihood Multi-Class Gradient

#### Single Element Derivative

Let's consider the derivative of the loss with respect to a single input $a_{k\ell}^L$. That is, we are looking for $\partial \mathrm{NLLM} / \partial a_{k\ell}^L $. Writing this in terms of the negative log-likelihood function,

$$ \frac{\partial}{\partial a_{k\ell}^L}\left(- \sum_{i=1}^{n^L}\sum_{j=1}^{b} y_{ij} \cdot \log a_{ij}^L\right) $$

This derivative is much easier to compute than the previous,

$$ \frac{\partial \mathrm{NLLM}}{\partial a_{k\ell}^L} = -\frac{y_{k\ell}}{a_{k\ell}^L} $$

#### Complete Gradient

Using the results from the previous section, we can construct a $2$ dimensional matrix containing the gradient with respect to each input

$$ \frac{\partial \mathrm{NLLM}}{\partial A^L} = \begin{pmatrix}
    -y_{11} / a_{11}^L & \cdots & -y_{1 b} / a_{1 b}^L \\
    \vdots & \ddots & \vdots \\
    -y_{n^L 1} / a_{n^L 1}^L & \cdots & -y_{n^L b} / a_{n^L b}^L
\end{pmatrix} $$

### Everything Together

We have a $3$-dimensional tensor and a $2$-dimensional matrix, how can we combine these to get the overall gradient of negative log-likelihood multi-class function with respect to the inputs to softmax for a batch? Remember, we are looking for,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} =\frac{\partial A^L}{\partial Z^L}\frac{\partial \mathrm{NLLM}}{\partial A^L} $$

To make this simpler, we can consider the case when we only have a single batch. That is, $b=1$ and $j = 1$. Then, this reduces to the matrix operation,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    a_1^L \left(1 - a_1^L\right) & -a_2^L a_1^L & \cdots & -a_{n^L}^L a_1^L \\
    -a_1^L a_2^L & a_2^L \left(1 - a_2^L\right) & \cdots & -a_{n^L}^L a_2^L \\
    \vdots & \vdots & \ddots & \vdots \\
    -a_1^L a_{n^L}^L & -a_2^L a_{n^L}^L & \cdots & a_{n^L}^L \left(1 - a_{n^L}^L\right)
\end{pmatrix} \cdot \begin{pmatrix}
    -y_{1} / a_{1}^L \\
    \vdots \\
    -y_{n^L} / a_{n^L}^L
\end{pmatrix} $$

If we have more than one batch, both of these matrices increase in a single dimension. To compute this weird product, we just consider layer-by-layer. That is,

$$ \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_{:j} = \begin{pmatrix}
    a_{1j}^L \left(1 - a_{1j}^L\right) & -a_{2j}^L a_{1j}^L & \cdots & -a_{n^Lj}^L a_{1j}^L \\
    -a_{1j}^L a_{2j}^L & a_{2j}^L \left(1 - a_{2j}^L\right) & \cdots & -a_{n^Lj}^L a_{2j}^L \\
    \vdots & \vdots & \ddots & \vdots \\
    -a_{1j}^L a_{n^Lj}^L & -a_{2j}^L a_{n^Lj}^L & \cdots & a_{n^Lj}^L \left(1 - a_{n^Lj}^L\right)
\end{pmatrix} \cdot \begin{pmatrix}
    -y_{1j} / a_{1j}^L \\
    \vdots \\
    -y_{n^Lj} / a_{n^Lj}^L
\end{pmatrix} $$

(Where the subscript $:j$ for the gradient represents taking the $j$ column from the matrix.)

### Standard Simplification

These matrices we have defined (and their product) are likely never computed directly in practice. Instead, relying on the fact that $Y$ is one-hot encoded, this whole result simplifies into a very simple closed form.

For now, consider the case with a single batch. Remember, we want to compute the following product,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    a_1^L \left(1 - a_1^L\right) & -a_2^L a_1^L & \cdots & -a_{n^L}^L a_1^L \\
    -a_1^L a_2^L & a_2^L \left(1 - a_2^L\right) & \cdots & -a_{n^L}^L a_2^L \\
    \vdots & \vdots & \ddots & \vdots \\
    -a_1^L a_{n^L}^L & -a_2^L a_{n^L}^L & \cdots & a_{n^L}^L \left(1 - a_{n^L}^L\right)
\end{pmatrix} \cdot \begin{pmatrix}
    -y_{1} / a_{1}^L \\
    \vdots \\
    -y_{n^L} / a_{n^L}^L
\end{pmatrix} $$

The result should be an $n^L \times 1$ vector. Let's take it element-by-element. The first element of this vector will be,

$$ \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_1 = \begin{pmatrix} a_1^L \left(1 - a_1^L\right) & -a_2^L a_1^L & \cdots & -a_{n^L}^L a_1^L \end{pmatrix} \cdot \begin{pmatrix}
    -y_{1} / a_{1}^L \\
    \vdots \\
    -y_{n^L} / a_{n^L}^L
\end{pmatrix} $$

Writing this out as a continued sum,

\begin{align*}
    \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_1 &= \left(a_1^L \left(1 - a_1^L\right)\right) \left(-\frac{y_{1}}{a_{1}^L}\right) + \left(-a_2^L a_1^L\right) \left(-\frac{y_{2}}{a_{2}^L}\right) + \ldots + \left(-a_{n^L}^L a_1^L\right) \left(-\frac{y_{n^L}}{a_{n^L}^L}\right) \\
    &= y_1 a_1^L - y_1 + y_2 a_1^L + \ldots + y_{n^L} a_1^L \\
    &= a_1^L \cdot \left(y_1 + y_2 + \ldots + y_{n^L}\right) - y_1
\end{align*}

Since we assume $Y$ is one-hot encoded, $\sum_{i = 1}^n y_i = 1$. Thus,

$$ \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_1 = a_1^L - y_1 $$

In complete matrix form, the gradient we are looking for is,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    a_1^L - y_1 \\
    \vdots \\
    a_{n^L}^L - y_{n^L}
    \end{pmatrix} $$
    
If we do not have a single batch, this result just expands into another dimension. That is,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    a_{11}^L - y_{11} & \cdots & a_{1b}^L - y_{1b} \\
    \vdots & \ddots & \vdots \\
    a_{n^L1}^L - y_{n^L 1} & \cdots & a_{n^L b}^L - y_{n^L b}
\end{pmatrix} $$

We will use this simple result to confirm that I have constructed the matrices correctly and multiply them together correctly.

## Numerically

Now this is all well and good, but how do we construct this $3$-dimensional tensor in practice? And then how do we multiply it with a $2$-dimensional matrix? We don't want to use a bunch of `for`-loops in Python (which would be a major performance-killer). This is where Einstein summation notation comes into play. If we can construct this matrix using a set of matrix prducts and sums (along arbitrary axes), then the `np.einsum` function can (more) efficiently compute the result using the C-compiled backend.

### The Environment

Let's set up the testing environment to make sure I am getting the proper results. Consider a situtation involving classification into three categories for a batch of two data points.

In [2]:
n, b = 3, 2

The predicted classifications are,

$$ A^L = \begin{pmatrix}
    0.2 & 0.5 \\
    0.5 & 0.1 \\
    0.3 & 0.4
\end{pmatrix} $$

The actual classifications are,

$$ Y = \begin{pmatrix}
    1.0 & 0.0 \\
    0.0 & 0.0 \\
    0.0 & 1.0
\end{pmatrix} $$

In [3]:
A = np.array([[0.2, 0.5],
              [0.5, 0.1],
              [0.3, 0.4]])

Y = np.array([[1.0, 0.0],
              [0.0, 0.0],
              [0.0, 1.0]])

Using this data, let's try to construct the tensor for $\partial A^L / \partial Z^L$. By hand, the result should be,

$$ \left(\frac{A^L}{Z^L}\right)_1 = \begin{pmatrix}
    0.2 \cdot (1 - 0.2) & -0.5 \cdot 0.2 & -0.3 \cdot 0.2 \\
    -0.2 \cdot 0.5 & 0.5 \cdot (1 - 0.5) & -0.3 \cdot 0.5 \\
    -0.2 \cdot 0.3 & -0.5 \cdot 0.3 & 0.3 \cdot (1 - 0.3)
\end{pmatrix} = \begin{pmatrix}
    0.16 & -0.10 & -0.06 \\
    -0.10 & 0.25 & -0.15 \\
    -0.06 & -0.15 & 0.21
\end{pmatrix} $$


$$ \left(\frac{A^L}{Z^L}\right)_2 = \begin{pmatrix}
    0.5 \cdot (1 - 0.5) & -0.1 \cdot 0.5 & -0.4 \cdot 0.5 \\
    -0.5 \cdot 0.1 & 0.1 \cdot (1 - 0.1) & -0.4 \cdot 0.1 \\
    -0.5 \cdot 0.4 & -0.1 \cdot 0.4 & 0.4 \cdot (1 - 0.4)
\end{pmatrix} = \begin{pmatrix}
    0.25 & -0.05 & -0.20 \\
    -0.05 & 0.09 & -0.04 \\
    -0.20 & -
    0.04 & 0.24
\end{pmatrix} $$

In [4]:
dAdZ = np.array([[[ 0.16, -0.10, -0.06],
                  [-0.10,  0.25, -0.15],
                  [-0.06, -0.15,  0.21]],
                
                 [[ 0.25, -0.05, -0.20],
                  [-0.05,  0.09, -0.04],
                  [-0.20, -0.04,  0.24]]])

# Since I want the batch dimension last, swap axes
dAdZ = dAdZ.swapaxes(0, 2)

Using the data, we can also construct the matrix for $\partial \mathrm{NLLM} / \partial A^L$. By hand, this comes out to be,

$$ \frac{\partial \mathrm{NLLM}}{\partial A^L} = \begin{pmatrix}
    -1.0 / 0.2 & -0.0 / 0.5 \\
    -0.0 / 0.5 & -0.0 / 0.1 \\
    -0.0 / 0.3 & -1.0 / 0.4
\end{pmatrix} = \begin{pmatrix}
    -5.0 & 0.0 \\
    0.0 & 0.0 \\
    0.0 & -2.5
\end{pmatrix} $$

In [5]:
dLdA = np.array([[-5.0,  0.0],
                 [ 0.0,  0.0],
                 [ 0.0, -2.5]])

We multiply these matrices together to get the final $\partial \mathrm{NLLM} / \partial Z^L$,

$$ \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_1 = \begin{pmatrix}
    0.16 & -0.10 & -0.06 \\
    -0.10 & 0.25 & -0.15 \\
    -0.06 & -0.15 & 0.21
\end{pmatrix} \cdot \begin{pmatrix}
    -5.0 \\
    0.0 \\
    0.0
\end{pmatrix} = \begin{pmatrix}
    -0.8 \\
    -0.5 \\
    -0.3
\end{pmatrix} $$

$$ \left(\frac{\partial \mathrm{NLLM}}{\partial Z^L}\right)_2 =  \begin{pmatrix}
    0.25 & -0.05 & -0.20 \\
    -0.05 & 0.09 & -0.04 \\
    -0.20 & -0.04 & 0.24
\end{pmatrix} \cdot \begin{pmatrix}
    0.0 \\
    0.0 \\
    -2.5
\end{pmatrix} = \begin{pmatrix}
    -0.5 \\
    -0.1 \\
    -0.6
\end{pmatrix} $$

Putting the two results together,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    -0.8 & 0.5 \\
    0.5 & 0.1 \\
    0.3 & -0.6
\end{pmatrix} $$

In [6]:
dAdZ[:,:,0] @ dLdA[:,0:1]

array([[-0.8],
       [ 0.5],
       [ 0.3]])

In [7]:
dAdZ[:,:,1] @ dLdA[:,1:2]

array([[ 0.5],
       [ 0.1],
       [-0.6]])

Using the simplified representation, we confirm our math is correct,

$$ \frac{\partial \mathrm{NLLM}}{\partial Z^L} = \begin{pmatrix}
    0.2 & 0.5 \\
    0.5 & 0.1 \\
    0.3 & 0.4
\end{pmatrix} - \begin{pmatrix}
    1.0 & 0.0 \\
    0.0 & 0.0 \\
    0.0 & 1.0
\end{pmatrix} =  \begin{pmatrix}
    -0.8 & 0.5 \\
    0.5 & 0.1 \\
    0.3 & -0.6
\end{pmatrix} $$

In [8]:
A - Y

array([[-0.8,  0.5],
       [ 0.5,  0.1],
       [ 0.3, -0.6]])

### Softmax Gradient

First, let's show how to construct the tensor $\partial A^L / \partial Z^L$ using nested `for`-loops.

In [9]:
dAdZ_ = np.zeros((n, n, b))

for i in range(n):    
    for j in range(n):        
        for k in range(b):            
            if i == j:
                dAdZ_[i, j, k] = A[i, k] * (1 - A)[i, k]
            else:
                dAdZ_[i, j, k] = -A[j, k] * A[i, k]

In [10]:
dAdZ_.swapaxes(0, 2)

array([[[ 0.16, -0.1 , -0.06],
        [-0.1 ,  0.25, -0.15],
        [-0.06, -0.15,  0.21]],

       [[ 0.25, -0.05, -0.2 ],
        [-0.05,  0.09, -0.04],
        [-0.2 , -0.04,  0.24]]])

Cool! So that is the matrix we are trying to create. However, we can't use conditionals in Einstein summation. We can construct the diagonal of the matrix using the following set of `for`-loops.

In [11]:
dAdZ_diag = np.zeros((n, n, b))

for i in range(n):    
    for j in range(n):        
        for k in range(b):            
            dAdZ_diag[i, j, k] = A[j, k] * (1 - A)[j, k] * np.eye(n)[j, i]

In [12]:
dAdZ_diag.swapaxes(0, 2)

array([[[0.16, 0.  , 0.  ],
        [0.  , 0.25, 0.  ],
        [0.  , 0.  , 0.21]],

       [[0.25, 0.  , 0.  ],
        [0.  , 0.09, 0.  ],
        [0.  , 0.  , 0.24]]])

Alright, so that is the diagonal we wish to use in our matrix. Let's see if we can encode this in Einstein summation notation.

In [13]:
dAdZ_diag = np.einsum('jk,jk,ji->ijk', A, 1 - A, np.eye(n))

In [14]:
dAdZ_diag.swapaxes(0, 2)

array([[[0.16, 0.  , 0.  ],
        [0.  , 0.25, 0.  ],
        [0.  , 0.  , 0.21]],

       [[0.25, 0.  , 0.  ],
        [0.  , 0.09, 0.  ],
        [0.  , 0.  , 0.24]]])

Perfect! So we now have the diagonal of our matrix. Let's see if we can construct the rest of the matrix using `for`-loops.

In [15]:
dAdZ_offdiag = np.zeros((n, n, b))

for i in range(n):    
    for j in range(n):        
        for k in range(b):            
            dAdZ_offdiag[i, j, k] = -A[j, k] * A[i, k] * (1 - np.eye(n))[j, i]

In [16]:
dAdZ_offdiag.swapaxes(0, 2)

array([[[-0.  , -0.1 , -0.06],
        [-0.1 , -0.  , -0.15],
        [-0.06, -0.15, -0.  ]],

       [[-0.  , -0.05, -0.2 ],
        [-0.05, -0.  , -0.04],
        [-0.2 , -0.04, -0.  ]]])

Alright, so that is the off-diagonal we wish to use in our matrix. Let's see if we can encode this in Einstein summation notation.

In [17]:
dAdZ_offdiag = np.einsum('jk,ik,ji->ijk', -A, A, 1 - np.eye(n))

In [18]:
dAdZ_offdiag.swapaxes(0, 2)

array([[[ 0.  , -0.1 , -0.06],
        [-0.1 ,  0.  , -0.15],
        [-0.06, -0.15,  0.  ]],

       [[ 0.  , -0.05, -0.2 ],
        [-0.05,  0.  , -0.04],
        [-0.2 , -0.04,  0.  ]]])

Beautiful! Now, we can just add these two together and be done.

In [19]:
dAdZ_ = dAdZ_diag + dAdZ_offdiag

In [20]:
dAdZ_.swapaxes(0, 2)

array([[[ 0.16, -0.1 , -0.06],
        [-0.1 ,  0.25, -0.15],
        [-0.06, -0.15,  0.21]],

       [[ 0.25, -0.05, -0.2 ],
        [-0.05,  0.09, -0.04],
        [-0.2 , -0.04,  0.24]]])

Now we can compare the methods to see which is more performant (and by how much). We start with the naive approach using three nested `for`-loops and a conditional statement.

In [21]:
def dAdZ_loop_conditional(A):
    """Constructs the dAdZ gradient tensor using nested for loops
    and a conditional statement.
    
    Args:
        A (ndarray): An n by b matrix of activations for a batch 
            of size b.
            
    Returns:
        ndarray: An n by n by b tensor representing the gradient
            of softmax activation with respect to inputs.
            
    """
    n, b = A.shape
    
    dAdZ = np.zeros((n, n, b))

    for i in range(n):
        for j in range(n):
            for k in range(b):
                if i == j:
                    dAdZ[i, j, k] = A[i, k] * (1 - A)[i, k]
                else:
                    dAdZ[i, j, k] = -A[j, k] * A[i, k]
    
    return dAdZ

In [38]:
%timeit dAdZ_loop_conditional(A).swapaxes(0, 2)

23.6 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Next, we consider the approach where we remove the conditional, but keep the `for`-loops.

In [23]:
def dAdZ_loop_nonconditional(A):
    """Constructs the dAdZ gradient tensor using nested for loops
    alone.
    
    Args:
        A (ndarray): An n by b matrix of activations for a batch 
            of size b.
            
    Returns:
        ndarray: An n by n by b tensor representing the gradient
            of softmax activation with respect to inputs.
            
    """
    n, b = A.shape
    
    dAdZ_diag = np.zeros((n, n, b))

    for i in range(n):
        for j in range(n):
            for k in range(b):            
                dAdZ_diag[i, j, k] = A[j, k] * (1 - A)[j, k] * np.eye(n)[j, i]

    dAdZ_offdiag = np.zeros((n, n, b))

    for i in range(n):
        for j in range(n):
            for k in range(b):            
                dAdZ_offdiag[i, j, k] = (-A)[j, k] * A[i, k] * (1 - np.eye(n))[j, i]
    
    return dAdZ_diag + dAdZ_offdiag

In [24]:
%timeit dAdZ_loop_nonconditional(A).swapaxes(0, 2)

234 µs ± 127 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Lastly, we consider the `np.einsum` implementation.

In [25]:
def dAdZ_einsum(A):
    """Constructs the dAdZ gradient tensor using np.einsum.
    
    Args:
        A (ndarray): An n by b matrix of activations for a batch 
            of size b.
            
    Returns:
        ndarray: An n by n by b tensor representing the gradient
            of softmax activation with respect to inputs.
            
    """
    return np.einsum('jk,jk,ji->ijk', A, 1 - A, np.eye(n)) \
           + np.einsum('jk,ik,ji->ijk', -A, A, 1 - np.eye(n))

In [37]:
%timeit dAdZ_einsum(A).swapaxes(0, 2)

19.6 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Thankfully, our work seems to have paid off--slightly. Although `np.einsum` is faster, it doesn't improve by that much. In practice, I'd probably construct the matrix using the `for`-loops with the conditional. It might be possible to reduce to a single call to `np.einsum` by making use of summation indices, but it is not obvious to me how to construct that.

### Negative Log-Likelihood Multi-Class Gradient

We can now construct the matrix $\partial \mathrm{NLLM} / \partial A^L$. This matrix is actually really easy to create. No fancy `np.einsum` is necessary; we can simply using broadcasting.

In [27]:
dLdA_ = -Y / A

In [28]:
dLdA_

array([[-5. , -0. ],
       [-0. , -0. ],
       [-0. , -2.5]])

### Everything Together

Now that we have both components of our gradient $\partial \mathrm{NLLM} / \partial Z^L$, we need to combine them. However, since we are trying to multiply a $3$-dimensional matrix with a $2$-dimensional matrix, we need to come up with a specialized method.

First, we can accomplish the desired behavior in a series of nested `for`-loops.

In [29]:
dLdZ_ = np.zeros((n, b))

for i in range(n):
    for j in range(b):
        
        total = 0
        
        for k in range(n):
            total += dAdZ[i, k, j] * dLdA[k, j]
        
        dLdZ_[i, j] = total

In [30]:
dLdZ_

array([[-0.8,  0.5],
       [ 0.5,  0.1],
       [ 0.3, -0.6]])

Now, let's see if we can encode this using `np.einsum`.

In [31]:
dLdZ_ = np.einsum('ikj,kj->ij', dAdZ, dLdA)

In [32]:
dLdZ_

array([[-0.8,  0.5],
       [ 0.5,  0.1],
       [ 0.3, -0.6]])

To continue the performance discussion, let's compare the loop version with the `np.einsum` version. Let's first consider the loop version.

In [33]:
def dLdZ_loop(dAdZ, dLdA):
    """Constructs the gradient of negative log-likelihood multi-class loss 
    with respect to the inputs of softmax activation for a single batch
    using nested loops.
        
    Args:
        dAdZ (ndarray): An n by n by b tensor representing the gradient of
            softmax activations with respect to softmax inputs for a batch
            of size b.
        dLdA (ndarray): An n by b matrix representing the gradient of
            negative log-likelihood multi-class loss with respect to the
            inputs of the loss for a batch of size b.
            
    Returns:
        ndarray: An n by b matrix representing the gradient of NLLM loss
            with respect to inputs of softmax for a batch of size b.
            
    """
    n, b = dLdA.shape
    
    dLdZ = np.zeros((n, b))

    for i in range(n):
        for j in range(b):

            total = 0

            for k in range(n):
                total += dAdZ[i, k, j] * dLdA[k, j]

            dLdZ[i, j] = total
    
    return dLdZ

In [34]:
%timeit dLdZ_loop(dAdZ, dLdA)

20.6 µs ± 6.53 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Next, let's consider the `np.einsum` implementation.

In [35]:
def dLdZ_einsum(dAdZ, dLdA):
    """Constructs the gradient of negative log-likelihood multi-class loss 
    with respect to the inputs of softmax activation for a single batch
    using np.einsum.
        
    Args:
        dAdZ (ndarray): An n by n by b tensor representing the gradient of
            softmax activations with respect to softmax inputs for a batch
            of size b.
        dLdA (ndarray): An n by b matrix representing the gradient of
            negative log-likelihood multi-class loss with respect to the
            inputs of the loss for a batch of size b.
            
    Returns:
        ndarray: An n by b matrix representing the gradient of NLLM loss
            with respect to inputs of softmax for a batch of size b.
            
    """
    return np.einsum('ikj,kj->ij', dAdZ, dLdA)

In [36]:
%timeit dLdZ_einsum(dAdZ, dLdA)

2.22 µs ± 50.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Now this is where `np.einsum` really shines. This is clearly the better method to construct this weird matrix product.